### Install modules
Note: You also might need to enter an API key from [Weights & Biases](https://wandb.ai/login). In that case, you should import the library with `import wandb` and then run `wandb.login(key="your_api_key_here")`


In [1]:
!pip install transformers datasets evaluate accelerate peft Pillow

### Imports

In [2]:
import torch
from transformers import RobertaTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, DataCollatorWithPadding
from peft import LoraConfig, get_peft_model, AutoPeftModelForSequenceClassification
from datasets import load_dataset, Dataset
from torch.utils.data import DataLoader
import evaluate
from tqdm import tqdm

c:\Users\Paulo Couto\anaconda3\envs\mva\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Model names

In [3]:
# Model and dataset configurations
peft_model_name = 'roberta-portuguese-peft'
modified_base = 'roberta-portuguese-modified'
base_model = 'roberta-base'

### Config Parameters

In [4]:
# Training params
n_epochs = 3
batch_size = 4
learning_rate = 5e-5

# LoRA params
lora_r = 8
lora_alpha = 16
lora_dropout = 0.1

### Get Dataset in Portuguese

In [5]:
# Load dataset
dataset = load_dataset('LIACC/Emakhuwa-Portuguese-News-MT')

Using the latest cached version of the dataset since LIACC/Emakhuwa-Portuguese-News-MT couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at C:\Users\Paulo Couto\.cache\huggingface\datasets\LIACC___emakhuwa-portuguese-news-mt\default\0.0.0\f9c6df57e9d419503712979dd3272e2bfc15bbfc (last modified on Tue Feb 18 20:27:46 2025).


### Example of instance in the Dataset

In [6]:
dataset['train'][0]

{'seg_index': 2,
 'pt': 'Matias Guente, do Canal de Moçambique, vence Prémio Internacional de Liberdade de Imprensa',
 'vmw': 'Matias Guene, ooKanaale ya Mocampiikhi, oolola e peremiyo internasionaale ya woopowa wa imprensa',
 'source': 'a_matias-guente-do-canal-de-moçambique-vence-prémio-internacional-de-liberdade-de-imprensa_5930109.txt',
 'project_title': 'desporto-cultura-1',
 'category': 'cultura',
 'domain': 'news',
 'writting_style': 'standard',
 'job_id': '6121524-e1d3e4d73a0f',
 'translators': 'Raja,benedito',
 'project_id': 6121524.0,
 'segment_id': 2591966325.0,
 'i_segment_id': 612152459301090.0}

### We'll use the `"pt"` column as input and `"category"` as label

In [7]:
print(f"Input text: {dataset['train'][0]['pt']}")
print(f"Label: {dataset['train'][0]['category']}")

Input text: Matias Guente, do Canal de Moçambique, vence Prémio Internacional de Liberdade de Imprensa
Label: cultura


### Merge all sets (train, test, validation) and filter by labels

In [8]:
# Define selected labels
selected_labels = {'cultura', 'desporto', 'economia', 'mundo', 'saude'}
num_labels = len(selected_labels)

# Merge all dataset splits into a single list
all_data = {
    'pt': dataset['train']['pt'] + dataset['validation']['pt'] + dataset['test']['pt'],
    'category': dataset['train']['category'] + dataset['validation']['category'] + dataset['test']['category']
}

# Filter dataset to only include selected labels
filtered_data = {
    'pt': [],
    'category': [],
    'labels': []
}

# Convert category names to numeric labels
label2id = {label: i for i, label in enumerate(sorted(selected_labels))}
id2label = {i: label for label, i in label2id.items()}

for text, category in zip(all_data['pt'], all_data['category']):
    if category in selected_labels:
        filtered_data['pt'].append(text)
        filtered_data['category'].append(category)
        filtered_data['labels'].append(label2id[category])

# Convert to Hugging Face Dataset
full_dataset = Dataset.from_dict(filtered_data)

# Shuffle dataset
full_dataset = full_dataset.shuffle(seed=42)

### Re-split the dataset into train, test and validation

In [9]:
# Split dataset (80% train, 10% validation, 10% test)
train_size = 0.8
val_size = 0.1
test_size = 0.1

train_test_split = full_dataset.train_test_split(test_size=(val_size + test_size), seed=42)
val_test_split = train_test_split['test'].train_test_split(test_size=(test_size / (val_size + test_size)), seed=42)

train_dataset = train_test_split['train']
eval_dataset = val_test_split['train']
test_dataset = val_test_split['test']

### Tokenize the datasets

In [10]:
# Initialize tokenizer
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenization function
def preprocess(examples):
    return tokenizer(examples['pt'], truncation=True, padding=True)

# Tokenize the datasets and remove unnecessary columns
train_dataset = train_dataset.map(preprocess, batched=True, remove_columns=['pt', 'category'])
eval_dataset = eval_dataset.map(preprocess, batched=True, remove_columns=['pt', 'category'])
test_dataset = test_dataset.map(preprocess, batched=True, remove_columns=['pt', 'category'])

# Print dataset sizes
print("\nDataset sizes:")
print(f"Train set: {len(train_dataset)}")
print(f"Eval set: {len(eval_dataset)}")
print(f"Test set: {len(test_dataset)}")

Map: 100%|██████████| 1435/1435 [00:01<00:00, 725.16 examples/s]


Dataset sizes:
Train set: 11478
Eval set: 1435
Test set: 1435


### Config Trainer

In [11]:
# Prepare data collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy='steps',
    learning_rate=learning_rate,
    num_train_epochs=n_epochs,
    per_device_train_batch_size=batch_size,
)

def get_trainer(model):
    return Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator,
    )

c:\Users\Paulo Couto\anaconda3\envs\mva\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### Initialize model

In [12]:
# Initialize the base model
model = AutoModelForSequenceClassification.from_pretrained(
    base_model,
    num_labels=num_labels,
    id2label=id2label,
    label2id=label2id
)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Config LoRA

In [13]:
# Configure and create PEFT model
peft_config = LoraConfig(
    task_type="SEQ_CLS",
    inference_mode=False,
    r=lora_r,
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout
)
peft_model = get_peft_model(model, peft_config)
print('PEFT Model')
peft_model.print_trainable_parameters()

PEFT Model
trainable params: 889,349 || all params: 125,538,826 || trainable%: 0.7084


### Train and Save

In [14]:
peft_lora_finetuning_trainer = get_trainer(peft_model)
peft_lora_finetuning_trainer.train()
peft_lora_finetuning_trainer.evaluate()

# Save the model and tokenizer
tokenizer.save_pretrained(modified_base)
peft_model.save_pretrained(peft_model_name)

Step,Training Loss,Validation Loss
500,1.474600,1.060661
1000,1.117800,0.961510
1500,1.081800,0.960028
2000,1.040800,0.962164
2500,1.047700,0.925508
3000,0.997700,0.915953
3500,0.985500,0.940330
4000,0.941300,0.891503
4500,0.982700,0.876724
5000,0.948700,0.863613


### Evaluation functions

In [15]:
# Inference functions
def load_model_for_inference():
    inference_model = AutoPeftModelForSequenceClassification.from_pretrained(
        peft_model_name,
        id2label=id2label
    )
    tokenizer = RobertaTokenizer.from_pretrained(modified_base)
    return inference_model, tokenizer

def classify(text, inference_model, tokenizer, gold):
    device = next(inference_model.parameters()).device  # Get the device the model is on
    inputs = tokenizer(text, truncation=True, padding=True, return_tensors="pt")
    # Move input tensors to the same device as the model
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        output = inference_model(**inputs)
    prediction = output.logits.argmax(dim=-1).item()
    emoji = "✅" if id2label[prediction] == gold else "❌"

    print(f'Text: {text}\nPrediction: {id2label[prediction]}\nGold: {gold}\n{emoji}\n')


# Evaluation function
def evaluate_model(inference_model, dataset):
    metric = evaluate.load('accuracy')
    eval_dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        collate_fn=data_collator
    )
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    inference_model.to(device)
    inference_model.eval()
    
    for batch in tqdm(eval_dataloader):
        batch.to(device)
        with torch.no_grad():
            outputs = inference_model(**batch)
            predictions = outputs.logits.argmax(dim=-1)
        metric.add_batch(
            predictions=predictions,
            references=batch["labels"]
        )
    
    eval_metric = metric.compute()
    return eval_metric

In [16]:
original_model = AutoModelForSequenceClassification.from_pretrained(
    base_model,
    num_labels=num_labels,
    id2label=id2label
)
base_perf = evaluate_model(original_model, test_dataset)

# Evaluate LoRA fine-tuned model
inference_model, tokenizer = load_model_for_inference()
lora_perf = evaluate_model(inference_model, test_dataset)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the latest cached version of the module from C:\Users\Paulo Couto\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--accuracy\f887c0aab52c2d38e1f8a215681126379eca617f96c447638f751434e8e65b14 (last modified on Fri Jan  3 17:49:54 2025) since it couldn't be found locally at evaluate-metric--accuracy, or remotely on the Hugging Face Hub.
100%|██████████| 359/359 [07:44<00:00,  1.29s/it]
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classif

### Compare test accuracies: Base vs LoRA

In [17]:
print(f"Base model performance: {base_perf['accuracy']:.3f}")
print(f"LoRA Fine-tuned model performance: {lora_perf['accuracy']:.3f}")

Base model performance: 0.190
LoRA Fine-tuned model performance: 0.702


### Try on some dummy examples created by me (native speaker)

### Original model

In [18]:
# Test classification
sample_text = "O chanceler se encontrou ontem com o Primeiro Ministro britânico para discutir as relações comerciais entre os dois países"
gold = "mundo"
classify(sample_text, original_model, tokenizer, gold)

sample_text = "O Banco Central decidiu aumentar novamente a taxa SELIC para tentar combater as altas da inflação"
gold = "economia"
classify(sample_text, original_model, tokenizer, gold)

sample_text = "O Cruzeiro anunciou a contratação de um novo treinador para o restante da temporada de 2025"
gold = "desporto"
classify(sample_text, original_model, tokenizer, gold)

sample_text = "O filme Ainda Estou aqui, que conta com a atriz Fernanda Torres, foi indicado a três premios, incluindo Melhor Filme"
gold = "cultura"
classify(sample_text, original_model, tokenizer, gold)

sample_text = "Jõao Fonseca ganhou ontem seu primeiro torneio na Argentina"
gold = "desporto"
classify(sample_text, original_model, tokenizer, gold)

sample_text = "Aumenta o número de casos de catapora e sarampo nas escolas da rede pública do país"
gold = "saude"
classify(sample_text, original_model, tokenizer, gold)

sample_text = "O Ministro da Fazenda estuda implementar uma taxa sobre importações vindas da China"
gold = "economia"
classify(sample_text, original_model, tokenizer, gold)

sample_text = "Representantes de Rússia e Estados Unidos se reuniram para debater o fim da guerra na Ucrânina"
gold = "mundo"
classify(sample_text, original_model, tokenizer, gold)

sample_text = "O surto de casos de dengue em São Paulo fez com que o governo intesificasse a campanha de vacinação"
gold = "saude"
classify(sample_text, original_model, tokenizer, gold)

Text: O chanceler se encontrou ontem com o Primeiro Ministro britânico para discutir as relações comerciais entre os dois países
Prediction: mundo
Gold: mundo
✅

Text: O Banco Central decidiu aumentar novamente a taxa SELIC para tentar combater as altas da inflação
Prediction: mundo
Gold: economia
❌

Text: O Cruzeiro anunciou a contratação de um novo treinador para o restante da temporada de 2025
Prediction: mundo
Gold: desporto
❌

Text: O filme Ainda Estou aqui, que conta com a atriz Fernanda Torres, foi indicado a três premios, incluindo Melhor Filme
Prediction: mundo
Gold: cultura
❌

Text: Jõao Fonseca ganhou ontem seu primeiro torneio na Argentina
Prediction: mundo
Gold: desporto
❌

Text: Aumenta o número de casos de catapora e sarampo nas escolas da rede pública do país
Prediction: mundo
Gold: saude
❌

Text: O Ministro da Fazenda estuda implementar uma taxa sobre importações vindas da China
Prediction: mundo
Gold: economia
❌

Text: Representantes de Rússia e Estados Unidos se reun

### LoRA model

In [19]:
# Test classification
sample_text = "O chanceler se encontrou ontem com o Primeiro Ministro britânico para discutir as relações comerciais entre os dois países"
gold = "mundo"
classify(sample_text, inference_model, tokenizer, gold)

sample_text = "O Banco Central decidiu aumentar novamente a taxa SELIC para tentar combater as altas da inflação"
gold = "economia"
classify(sample_text, inference_model, tokenizer, gold)

sample_text = "O Cruzeiro anunciou a contratação de um novo treinador para o restante da temporada de 2025"
gold = "desporto"
classify(sample_text, inference_model, tokenizer, gold)

sample_text = "O filme Ainda Estou aqui, que conta com a atriz Fernanda Torres, foi indicado a três premios, incluindo Melhor Filme"
gold = "cultura"
classify(sample_text, inference_model, tokenizer, gold)

sample_text = "Jõao Fonseca ganhou ontem seu primeiro torneio na Argentina"
gold = "desporto"
classify(sample_text, inference_model, tokenizer, gold)

sample_text = "Aumenta o número de casos de catapora e sarampo nas escolas da rede pública do país"
gold = "saude"
classify(sample_text, inference_model, tokenizer, gold)

sample_text = "O Ministro da Fazenda estuda implementar uma taxa sobre importações vindas da China"
gold = "economia"
classify(sample_text, inference_model, tokenizer, gold)

sample_text = "Representantes de Rússia e Estados Unidos se reuniram para debater o fim da guerra na Ucrânina"
gold = "mundo"
classify(sample_text, inference_model, tokenizer, gold)

sample_text = "O surto de casos de dengue em São Paulo fez com que o governo intesificasse a campanha de vacinação"
gold = "saude"
classify(sample_text, inference_model, tokenizer, gold)

Text: O chanceler se encontrou ontem com o Primeiro Ministro britânico para discutir as relações comerciais entre os dois países
Prediction: mundo
Gold: mundo
✅

Text: O Banco Central decidiu aumentar novamente a taxa SELIC para tentar combater as altas da inflação
Prediction: economia
Gold: economia
✅

Text: O Cruzeiro anunciou a contratação de um novo treinador para o restante da temporada de 2025
Prediction: desporto
Gold: desporto
✅

Text: O filme Ainda Estou aqui, que conta com a atriz Fernanda Torres, foi indicado a três premios, incluindo Melhor Filme
Prediction: cultura
Gold: cultura
✅

Text: Jõao Fonseca ganhou ontem seu primeiro torneio na Argentina
Prediction: desporto
Gold: desporto
✅

Text: Aumenta o número de casos de catapora e sarampo nas escolas da rede pública do país
Prediction: saude
Gold: saude
✅

Text: O Ministro da Fazenda estuda implementar uma taxa sobre importações vindas da China
Prediction: economia
Gold: economia
✅

Text: Representantes de Rússia e Estados 